In [0]:
from pyspark.sql.functions import col, split, explode, expr, monotonically_increasing_id

In [0]:
spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "text") \
    .load("/Volumes/workspace/dev/trading/") \
.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/workspace/dev/trading/checkpoint/bronze") \
    .trigger(availableNow=True) \
    .table("workspace.default.bronze_fix_messages")

In [0]:
%sql
SELECT * FROM workspace.default.bronze_fix_messages

In [0]:
spark.readStream \
    .table("workspace.default.bronze_fix_messages") \
    .filter(col("value").startswith("8=")) \
.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/workspace/dev/trading/checkpoint/silver") \
    .trigger(availableNow=True) \
    .table("workspace.default.silver_fix_messages")

In [0]:
%sql
SELECT * FROM workspace.default.silver_fix_messages